<a href="https://colab.research.google.com/github/ChrisKantor/Deep-Learning/blob/main/Project%201/Deep_Learning_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Deep Learning model that can identify the state of different cooking ingredients

#submit accuracy/loss figures for each epoch for both training and validation data

#https://www.tensorflow.org/tutorials/images/classification

#focus on getting a working model for pt 2. try a simple one first, then gradually increase complexity until we acheive overfitting. submit accuracy/loss figures for this part
#pt3 is all about hyperparameter tuning/data augmentation

# Loading Data

In [1]:
#import libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import os
from matplotlib import pyplot as plt
import PIL

## Mount Google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data_dir = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/Project 1/data/'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Deep Learning/Project 1/data/


In [10]:
#Some hyperparameter setup
image_size = (256, 256)
batch_size = 32
label_mode = 'categorical' #uses categorical_crossentropy , 'int' uses sparse_categorical_crossentropy

In [11]:
#load data set
print(data_dir)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + 'train/',
    image_size = image_size,
    batch_size = batch_size,
    label_mode = label_mode,
    seed = 8312001
)

print(train_ds.class_names)

/content/drive/MyDrive/Colab Notebooks/Deep Learning/Project 1/data/
Found 7213 files belonging to 11 classes.
['creamy_paste', 'diced', 'floured', 'grated', 'juiced', 'jullienne', 'mixed', 'other', 'peeled', 'sliced', 'whole']
